In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from collections import Counter
import json
import subprocess

print(tf.__version__)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

2025-10-08 21:35:54.251462: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.16.2
1 Physical GPUs, 1 Logical GPUs


2025-10-08 21:35:56.469777: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:926] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-10-08 21:36:00.064968: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:926] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-10-08 21:36:00.065003: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:926] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-10-08 21:36:00.065987: I external/local_xla/xla/stream_executor/rocm/rocm_executor.cc:926] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-10-08 21:36:00.066027: I external/local_xla/xla/stream_executor/rocm/rocm_executor.

In [3]:
# clean for debug purpose only 

trainData = None
valData = None
testData = None
model = None

tf.keras.backend.clear_session(free_memory=True)


## Loading the data

TODO

In [2]:
# load images and labels 


labelSet = Counter()
dataDict = {}
vocab = set()


datasetLen = 0
with open("./A2_train_v3.jsonl", "r") as jsonFile:
	for line in jsonFile:
		datasetLen += 1
		loadedLine = json.loads(line)
		if loadedLine["Image_ID"] not in dataDict:
			dataDict[loadedLine["Image_ID"]] = []

		labelSet[loadedLine["Label"]] += 1

		hypo = [''.join(char for char in word if char.isalnum()) for word in loadedLine["Hypothesis"].lower().split()]
		vocab.update(hypo)

		dataDict[loadedLine["Image_ID"]].append((hypo, loadedLine["Label"]))

labelTuple = tuple(labelSet.keys())
vocab = list(vocab)

vocabIndex = {vocab[i]: i for i in range(len(vocab))}

print(len(dataDict.keys()))
print(len(vocabIndex))
print(datasetLen)
print(labelSet)

19573
9274
39129
Counter({'entailment': 19619, 'contradiction': 19510})


## Creating the tensorflow dataset

TODO

In [3]:
# load the glove embeddings 

def getGlove():
  print('Downloading glove')
  subprocess.run(['wget', 'https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip'])
  subprocess.run(['unzip', '-q glove.6B.zip'])

def generateMatrix(dim):
    print('parsing glove data')
    embeddingMatrix = np.zeros((len(vocab), dim))
    embeddedVocab = []
    
    with open(f'glove.6B.{dim}d.txt', encoding="utf-8") as gloveFile:
      for line in gloveFile:
        values = line.split()
        word = values[0]
    
        if word in vocab:
          embeddingMatrix[vocabIndex[word]] = np.asarray(values[1:], dtype='float32')
          embeddedVocab.append(word)

    print(f'embedded {len(embeddedVocab)} out of {len(vocab)}')
    return embeddingMatrix


embeddingDim = 200
embeddingMatrix = None

if not os.path.isfile(f'glove.6B.{embeddingDim}d.txt'):
  getGlove()

if os.path.isfile(f'embeddingMatrix.{embeddingDim}d.pkl'):
  with open(f'embeddingMatrix.{embeddingDim}d.pkl', 'rb') as f:
    embeddingMatrix = pickle.load(f)

  print('loaded matrix')
else:
  embeddingMatrix = generateMatrix(embeddingDim)

  with open(f'embeddingMatrix.{embeddingDim}d.pkl', 'ab') as f:
    pickle.dump(embeddingMatrix, f)

  print('saved matrix')

loaded matrix


In [157]:
maxLen = 128

X1array = []
X2array = []
YArray = []

for key, hypoAndLabels in dataDict.items():
	img = f'./A2_Images/{key}.jpg'

	for hypo, label in hypoAndLabels:
		label = tf.convert_to_tensor([labelTuple.index(label)])
		label.set_shape([1])

		hypo = [vocabIndex[word] for word in hypo]
		hypo = tf.convert_to_tensor(tf.keras.preprocessing.sequence.pad_sequences([hypo], maxlen=maxLen)[0])
		hypo.set_shape([maxLen])

		X1array.append(img)
		X2array.append(hypo)
		YArray.append(label)

X1Numpy = np.array(X1array)
X2Numpy = np.array(X2array, dtype='uint16')
YNumpy = np.array(YArray, dtype='uint8')

dataset = tf.data.Dataset.from_tensor_slices(({'image': X1Numpy, 'text': X2Numpy}, YNumpy))

imageChannels = 3
imageRes = 256
patchesPerImage = 8

patchRes = int(imageRes / patchesPerImage) 
patchNum = patchesPerImage ** 2 * imageChannels 

def getImage(path):
	img = tf.io.read_file(path)
	img = tf.io.decode_image(img, channels=imageChannels, dtype=tf.float32)
	img = tf.image.resize(img, (imageRes, imageRes))
	img = tf.reshape(img, (1, imageRes, imageRes, imageChannels))
	img = tf.image.extract_patches(
		images=img,
		sizes=[1, patchesPerImage, patchesPerImage, 1],
		strides=[1, patchesPerImage, patchesPerImage, 1],
		rates=[1, 1, 1, 1],
		padding='VALID'
	)
	img = tf.reshape(img, (patchRes ** 2, patchNum))

	return img

def getImageWrapper(x, y):
	img = tf.py_function(func=getImage, inp=[x['image']], Tout=tf.float32)
	img.set_shape([patchRes ** 2, patchNum,])

	x['image'] = img 
	return x, y

dataset = dataset.map(getImageWrapper, num_parallel_calls=tf.data.AUTOTUNE)

In [158]:
print(patchRes)
print(patchNum)

for i in dataset.take(1):
	print(i)

32
192
({'image': <tf.Tensor: shape=(1024, 192), dtype=float32, numpy=
array([[0.21750155, 0.45987827, 0.6347683 , ..., 0.19901885, 0.45414293,
        0.6514545 ],
       [0.16561824, 0.43893322, 0.6393315 , ..., 0.8262642 , 0.7975571 ,
        0.72094053],
       [0.829873  , 0.82487917, 0.748381  , ..., 0.81371564, 0.78626466,
        0.7117548 ],
       ...,
       [0.12718633, 0.13094592, 0.13310924, ..., 0.5203906 , 0.17092675,
        0.26866114],
       [0.11916474, 0.11280201, 0.14267261, ..., 0.70257074, 0.22066449,
        0.27169263],
       [0.64206034, 0.50968987, 0.4923476 , ..., 0.5946818 , 0.5394631 ,
        0.50480247]], dtype=float32)>, 'text': <tf.Tensor: shape=(128,), dtype=uint16, numpy=
array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0, 

In [156]:
testSize = int(datasetLen * 0.05)
valSize = int(datasetLen * 0.05)
trainSize = int(datasetLen - testSize - valSize)
batchSize = 100

def optimize(ds):
	ds = ds.batch(batchSize) 
	ds = ds.cache() 
	ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
	
	return ds

def getTest(ds):
	ds = ds.take(testSize) 
	ds = optimize(ds)
	
	return ds

def getVal(ds):
	ds = ds.skip(testSize)
	ds = ds.take(valSize) 
	ds = optimize(ds)

	return ds

def getTrain(ds):
	ds = ds.skip(valSize + testSize)
	ds = ds.take(trainSize)
	ds = optimize(ds)

	return ds

testDS = getTest(dataset)
valDS = getVal(dataset)
trainDS = getTrain(dataset)


print(f"test data batches {tf.data.experimental.cardinality(testDS).numpy()}")
print(f"val data batches {tf.data.experimental.cardinality(valDS).numpy()}")
print(f"train data batches {tf.data.experimental.cardinality(trainDS).numpy()}")
print(f'ratios test:{testSize} val:{valSize} train:{trainSize}')


test data batches 20
val data batches 20
train data batches 353
ratios test:1956 val:1956 train:35217


# Training

ToDo


In [161]:
denseStrides = 64

class PositionEncoder(tf.keras.Layer):
	def __init__(self, patchRes, patchNum):
		super(PositionEncoder, self).__init__()
		self.patchRes = patchRes
		self.patchNum = patchNum
		w_init = tf.random_normal_initializer()
		classToken = w_init(shape=(1, patchNum), dtype=tf.float32)
		self.classToken = tf.Variable(initial_value=classToken, trainable=True)
		self.projection =tf.keras.layers.Dense(units=patchNum)
		self.positionEmbedding = tf.keras.layers.Embedding(input_dim=patchRes+1, output_dim=patchNum, mask_zero=True)

	def call(self, patch):
		batch = tf.shape(patch)[0]
		classToken = tf.tile(self.classToken, multiples = [batch, 1])
		classToken = tf.reshape(classToken, (batch, 1, self.patchNum))
		patchesEmbed = self.projection(patch)
		patchesEmbed = tf.concat([patchesEmbed, classToken], 1)
		positions = tf.range(start=0, limit=self.patchRes+1, delta=1)
		positionsEmbed = self.positionEmbedding(positions)
		encoded = patchesEmbed + positionsEmbed
		return encoded

class Transformer(tf.keras.Layer):
	def __init__(self, dim, denseSize, heads=8, dropout=0.1):
		super(Transformer, self).__init__()
		self.norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
		self.attn = tf.keras.layers.MultiHeadAttention(num_heads=heads, key_dim=dim, dropout=dropout)
		self.norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
		self.dense1 = tf.keras.layers.Dense(denseSize, activation='relu')
		self.dense2 = tf.keras.layers.Dense(dim, activation='relu')
		self.drop = tf.keras.layers.Dropout(dropout)

	def call(self, inputLayers):
		attention = self.attn(inputLayers, inputLayers)
		attention = tf.keras.layers.Add()([attention, inputLayers])
		attention = self.norm1(attention)

		mlp = self.drop(attention)
		mlp = self.dense1(mlp)
		mlp = self.drop(mlp)
		mlp = self.dense2(mlp)
		mlp = tf.keras.layers.Add()([mlp, attention])
		mlp = self.norm2(mlp)

		return mlp

# Image Encoder 

inputImageLayer = tf.keras.layers.Input(shape=(patchRes**2, patchNum), name='image')
embeddingImage = PositionEncoder(patchRes**2, patchNum)(inputImageLayer)

transformerImage = Transformer(patchNum, 128)(embeddingImage)

imageOut = tf.keras.layers.Dense(denseStrides, activation='relu')(transformerImage)


# Text Encoder 
inputTextLayer = tf.keras.layers.Input(shape=(maxLen,), name='text')

embeddingText = tf.keras.layers.Embedding(len(vocab), embeddingDim, weights=[embeddingMatrix], trainable=False, name='embedding')(inputTextLayer)
reshapeText = tf.keras.layers.Reshape((embeddingDim, maxLen))(embeddingText)
embeddingPositionText = PositionEncoder(embeddingDim, maxLen)(reshapeText)

transformerText = Transformer(maxLen, 128)(embeddingPositionText)

textOut = tf.keras.layers.Dense(denseStrides, activation='relu')(attentionText)

# Entanglement decoder 
decoderInput = tf.keras.layers.Concatenate(-2, name='decoderInput')([imageOut, textOut])

transformerDecode = Transformer(denseStrides, 128)(decoderInput)

flat = tf.keras.layers.GlobalAveragePooling1D()(transformerDecode)
dense = tf.keras.layers.Dense(256, activation='relu')(flat)
decoderOutput = tf.keras.layers.Dense(1, name='output', activation='sigmoid')(dense)

model = tf.keras.Model(inputs=inputImageLayer, outputs=decoderOutput)

model.compile(
  optimizer='adam',
  loss='binary_crossentropy',
  metrics=['accuracy']
)
model.summary()


Model: "functional_21"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ text (InputLayer)   │ (None, 128)       │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 128, 200)  │  1,854,800 │ text[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ image (InputLayer)  │ (None, 1024, 192) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_10          │ (None, 200, 128)  │          0 │ embedding[0][0]   │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ position_encoder_54 │ (None, 1025, 192) │    233,856 │ image[0][0]       │
│ (PositionEncoder)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ position_encoder_19 │ (None, 201, 128)  │     42,240 │ reshape_10[0][0]  │
│ (PositionEncoder)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_26      │ (None, 1025, 192) │  1,234,688 │ position_encoder… │
│ (Transformer)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 201, 128)  │    527,488 │ position_encoder… │
│ (MultiHeadAttentio… │                   │            │ position_encoder… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_194 (Dense)   │ (None, 1025, 64)  │     12,352 │ transformer_26[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_198 (Dense)   │ (None, 201, 64)   │      8,256 │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoderInput        │ (None, 1226, 64)  │          0 │ dense_194[0][0],  │
│ (Concatenate)       │                   │            │ dense_198[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ transformer_28      │ (None, 1226, 64)  │    149,504 │ decoderInput[0][… │
│ (Transformer)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ transformer_28[0… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_201 (Dense)   │ (None, 256)       │     16,640 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │        257 │ dense_201[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,080,081 (15.56 MB)

 Trainable params: 2,225,281 (8.49 MB)

 Non-trainable params: 1,854,800 (7.08 MB)

In [166]:
# Training

history = model.fit(
  trainDS,
	validation_data=valDS,
  epochs=8,
  batch_size=batchSize
)

Epoch 1/8


KeyError: "Exception encountered when calling Functional.call().\n\n\x1b[1m136784566597280\x1b[0m\n\nArguments received by Functional.call():\n  • inputs={'image': 'tf.Tensor(shape=(None, 1024, 192), dtype=float32)', 'text': 'tf.Tensor(shape=(None, 128), dtype=uint16)'}\n  • training=True\n  • mask={'image': 'None', 'text': 'None'}\n  • kwargs=<class 'inspect._empty'>"

In [ ]:
model.save('./final.keras')